<a href="https://colab.research.google.com/github/malekkhammassi/A-CNN-based-Stereo-Image-Denoising-Method/blob/master/Stereo_Image_Denoising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **The implementation is based on the framework Pytorch and the network was trained and tested using Google Colab.**

### 1.   Mounting the Drive
This process is necessary to connect google drive to google colab and therefore to load the data into google colab.






In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2.   Preparing the Environment
In order to prepare the suitable environment for the code, we provide a requirements.txt file on our github project.


1.   Download the requirements file or copy/paste its content on your laptop.
2.   Upload the file by executing the next cell.




In [3]:
# upload the requirements file
from google.colab import files 
files.upload()

Saving requirements.txt to requirements.txt


{'requirements.txt': b'opencv-python==3.4.1.15\nscikit-image==0.12.3\ntorch==0.4.0\ntorchvision==0.2.1\nh5py==2.7.0\n\n\nnumpy==1.14.4'}

The next step is to install the requirements by executing the next four cells.

In [0]:
#install the requirements
!pip install -r requirements.txt

In [0]:
!pip install numpy --upgrade

In [0]:
!pip install scikit-image --upgrade

In [0]:
!git clone https://github.com/lanpa/tensorboardX && cd tensorboardX && python setup.py install

### 3. Installing the Necessary Libreries 



In [0]:
import math
import torch
import torch.nn as nn
import os
import os.path
import random
import h5py
import torch
import glob
import torch.utils.data as udata
import re
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from statistics import mean
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
from skimage.measure import compare_psnr, compare_ssim

The next cell contains necessary functions needed during training and testing:
1. **weights_init_kaiming** initializes the weights of the networks in the first epoch of training.
2. **batch_PSNR** Computes the PSNR of a batch of 128 images.
3. **batch_SSIM** Computes the SSIM of a batch of 128 images.
4. **findLastCheckpoint** finds the last epoch of training.
5. **data_augmentation** applies data augmentation on one image.


In [0]:


def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode='fan_in')
    elif classname.find('Linear') != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode='fan_in')
    elif classname.find('BatchNorm') != -1:
        # nn.init.uniform(m.weight.data, 1.0, 0.02)
        m.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).clamp_(-0.025,0.025)
        nn.init.constant(m.bias.data, 0.0)

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        im1 = Iclean[i,:,:,:].transpose((1,2,0))
        im2 =  Img[i,:,:,:].transpose((1,2,0))
        PSNR += compare_psnr(im1, im2)
    return (PSNR/Img.shape[0])

def batch_SSIM(img, imclean):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    SSIM = 0
    for i in range(Img.shape[0]):
        im1 = Iclean[i,:,:,:].transpose((1,2,0))
        im2 =  Img[i,:,:,:].transpose((1,2,0))
        SSIM += compare_ssim(im1, im2, multichannel=True)
    return ( SSIM/Img.shape[0])
  
def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir, 'model_*.pth'))
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).pth.*", file_)
            epochs_exist.append(int(result[0]))
        initial_epoch = max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch

def data_augmentation(image, mode):
    out = np.transpose(image, (1,2,0))
    if mode == 0:
        # original
        out = out
    elif mode == 1:
        # flip up and down
        out = np.flipud(out)
    elif mode == 2:
        # rotate counterwise 90 degree
        out = np.rot90(out)
    elif mode == 3:
        # rotate 90 degree and flip up and down
        out = np.rot90(out)
        out = np.flipud(out)
    elif mode == 4:
        # rotate 180 degree
        out = np.rot90(out, k=2)
    elif mode == 5:
        # rotate 180 degree and flip
        out = np.rot90(out, k=2)
        out = np.flipud(out)
    elif mode == 6:
        # rotate 270 degree
        out = np.rot90(out, k=3)
    elif mode == 7:
        # rotate 270 degree and flip
        out = np.rot90(out, k=3)
        out = np.flipud(out)
    return np.transpose(out, (2,0,1))


# **Model**
The next cell contains the architecture of our model.

In [0]:
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []
        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
    
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_of_layers-2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)
    def forward(self, x):
        out = self.dncnn(x)
        return out

# **Dataset**
The next cell prepares the data for the training:
1. **normalize** applies normalization on the images.
2. **Im2Patch** prepares the patches.
3. **prepare_data** contructs a 3D image out of the two views (left and right), it calls Im2Patch, data_augmentation and normalize to prepare the training patches and it also prepares the validation data.

In [0]:
def normalize(data):
    return data/255.

def Im2Patch(img, win, stride=1):
    k = 0
    endc = img.shape[2]
    endw = img.shape[1]
    endh = img.shape[0]
    patch = img[0:endw-win+0+1:stride, 0:endh-win+0+1:stride,:]
    TotalPatNum = patch.shape[1] * patch.shape[0]
    Y = np.zeros([endc, win*win,TotalPatNum], np.float32)
    for i in range(win):
        for j in range(win):
            patch = img[i:endw-win+i+1:stride,j:endh-win+j+1:stride,:]
            Y[:,k,:] = np.array(np.resize(patch[:],(endc, TotalPatNum)))
            k = k + 1
    return Y.reshape([endc, win, win, TotalPatNum])

def prepare_data(data_path, patch_size, stride, aug_times=1):
    # training data
    print('process training data')
    scales = [1]
    files_right = glob.glob(os.path.join(data_path, 'train_right', '*.png'))
    files_left = glob.glob(os.path.join(data_path, 'train_left', '*.png'))
    files_right.sort()
    files_left.sort()
    h5f = h5py.File(os.path.join(data_path, 'train.h5'), 'w')
    train_num = 0
    for i in range(len(files_right)):
        img_right = cv2.imread(files_right[i],0)
        img_left = cv2.imread(files_left[i],0)
        img_right= np.expand_dims(img_right,2)
        img_left = np.expand_dims(img_left,2)
        hr, wr, cr = img_right.shape
        img=np.zeros((hr,wr,2))
        img[:,:,0]=img_right[:,:,0]
        img[:,:,1]=img_left[:,:,0]
        h, w, c = img.shape #img is a 3D image that has the right view on the first channel and the left view on the second channel
        for k in range(len(scales)):
            Img = cv2.resize(img, (int(h*scales[k]), int(w*scales[k])), interpolation=cv2.INTER_CUBIC)
            Img = np.float32(normalize(Img))
            patches = Im2Patch(Img, win=patch_size, stride=stride)
            print("file: %s scale %.1f # samples: %d" % (files_right[i], scales[k], patches.shape[3]*aug_times))
            for n in range(patches.shape[3]):
                data = patches[:,:,:,n].copy()
                h5f.create_dataset(str(train_num), data=data)
                train_num += 1
                for m in range(aug_times-1):
                    data_aug = data_augmentation(data, np.random.randint(1,8))
                    h5f.create_dataset(str(train_num)+"_aug_%d" % (m+1), data=data_aug)
                    train_num += 1
    h5f.close()
    # validation data
    print('\nprocess validation data')
    files_right.clear()
    files_left.clear()
    files_right = glob.glob(os.path.join(data_path, 'validation_right', '*.png'))
    files_left  = glob.glob(os.path.join(data_path, 'validation_left' , '*.png'))
    files_right.sort()
    files_left.sort()
    h5f = h5py.File(os.path.join(data_path, 'val.h5'), 'w')
    val_num = 0
    for i in range(len(files_right)):
        print("files: ", files_right[i], files_left[i])
        img_right = cv2.imread(files_right[i],0)
        img_left  = cv2.imread(files_left [i],0)
        img_right= np.expand_dims(img_right,2)
        img_left = np.expand_dims(img_left,2)
        hv, wv, cv= img_right.shape
        img=np.zeros((2,hv,wv))
        img[0,:,:]=img_right[:,:,0]
        img[1,:,:]=img_left[:,:,0]
        img = np.float32(normalize(img)) #img is a 3D image that has the right view on the first channel and the left view on the second channel
        h5f.create_dataset(str(val_num), data=img)
        val_num += 1
    h5f.close()
    print('training set, # samples %d\n' % train_num)
    print('val set, # samples %d\n' % val_num)

class Dataset(udata.Dataset):
    def __init__(self, train=True):
        super(Dataset, self).__init__()
        self.train = train
        if self.train:
            h5f = h5py.File(os.path.join(data_path, 'train.h5') ,'r')
        else:
            h5f = h5py.File(os.path.join(data_path, 'val.h5'), 'r')
        self.keys = list(h5f.keys())
        random.shuffle(self.keys)
        h5f.close()
    def __len__(self):
        return len(self.keys)
    def __getitem__(self, index):
        if self.train:
            h5f = h5py.File(os.path.join(data_path, 'train.h5') , 'r')
        else:
            h5f = h5py.File(os.path.join(data_path, 'val.h5') , 'r')
        key = self.keys[index]
        data = np.array(h5f[key])
        h5f.close()
        return torch.Tensor(data)


# **Training**
The hyperparamters are defined at the beginning of the code:

**preprocess** = False if the data is already preprocessed.

**batchSize**     = 128   is the batch size.

**num_of_layers** = 17    is the number of layers.

**epochs**        = 50    is the number of the training epochs.

**milestone**     = 30    is the number of milestone (the number of epochs where we change the learning rate).

**lr** = 1e-3  is the learning rate.

**outf** = "/content/drive/My Drive/Stereo-PyTorch-master/logs/new_training2" is the file that will contain the trained models. 

**mode** is "S" is the mode is "S" if the training is for a specific noise level and it is "B" if the training is for blind denoising.

**noiseL** = 35  is the noise level during training.

**val_noiseL** = 35 is the noise level during validation.

In [0]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

preprocess    = True  #preprocess = False if the data is already preprocessed
batchSize     = 128   #the batch size
num_of_layers = 17    #the number of layers
epochs        = 50    #the number of the training epochs
milestone     = 30    #the number of milestone (the number of epochs where we change the learning rate)
lr            = 1e-3  #the learning rate
outf          = "/content/drive/My Drive/Stereo-PyTorch-master/logs/new_training" # the file of containig the trained models 
mode          = "S" # the mode is "S" if the training is for a specific noise level and it is "B" if the training is for blind denoising.
noiseL        = 35  # the noise level during training.
val_noiseL    = 35  # the noise level during validation.
data_path     = '/content/drive/My Drive/Stereo-PyTorch-master/data'

def main():
    # Load dataset
    print('Loading dataset ...\n')
    dataset_train = Dataset(train=True)
    dataset_val = Dataset(train=False)
    loader_train = DataLoader(dataset=dataset_train, num_workers=4, batch_size=batchSize, shuffle=True)
    print("# of training samples: %d\n" % int(len(dataset_train)))
    
    # Build model and find last check point
    net = DnCNN(channels=2, num_of_layers=num_of_layers)
    initial_epoch = findLastCheckpoint(save_dir=outf)

    # Loading the last trained model and the previous values of PSNR and Loss
    if initial_epoch > 0:
      print('resuming by loading epoch %03d' % initial_epoch)
      net = torch.load(os.path.join(outf , 'model_%03d.pth' % initial_epoch))
      loss_epoch_train = np.load(os.path.join(outf,'loss_epoch_train.npy'))
      loss_epoch_val = np.load(os.path.join(outf,'loss_epoch_val.npy'))
      PSNR_epoch_train = np.load(os.path.join(outf,'PSNR_epoch_train.npy'))
      PSNR_epoch_val = np.load(os.path.join(outf,'PSNR_epoch_val.npy'))
    else:
      print('we are starting from the first epoch')
      net.apply(weights_init_kaiming)
      loss_epoch_train = []
      loss_epoch_val = []
      PSNR_epoch_train = []
      PSNR_epoch_val = []
      
    criterion = nn.MSELoss(size_average=False)
    
    # Move to GPU
    device_ids = [0]
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    criterion.cuda()
    
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    # training
    writer = SummaryWriter(outf)
    step = 0
    noiseL_B=[0,55] # ingnored when opt.mode=='S'
    for epoch in range(initial_epoch, epochs):
        if epoch < milestone:
            current_lr = lr
        else:
            current_lr = lr / 10.
        # set learning rate
        for param_group in optimizer.param_groups:
            param_group["lr"] = current_lr
        print('learning rate %f' % current_lr)
        
        
        # train
        loss_train=[]
        PSNR_train=[]
        for i, data in enumerate(loader_train, 0):
            # training step
            model.train()
            model.zero_grad()
            optimizer.zero_grad()
            img_train = data
            if mode == 'S':
                noise = torch.FloatTensor(img_train.size()).normal_(mean=0, std=noiseL/255.)
                
            if mode == 'B':
                noise = torch.zeros(img_train.size())
                stdN = np.random.uniform(noiseL_B[0], noiseL_B[1], size=noise.size()[0])
                for n in range(noise.size()[0]):
                    sizeN = noise[0,:,:,:].size()
                    noise[n,:,:,:] = torch.FloatTensor(sizeN).normal_(mean=0, std=stdN[n]/255.)
            imgn_train = img_train + noise
            img_train, imgn_train = Variable(img_train.cuda()), Variable(imgn_train.cuda())
            noise = Variable(noise.cuda())
            out_train = model(imgn_train)
            loss = criterion(out_train, noise) / (imgn_train.size()[0]*2)
            loss.backward()
            optimizer.step()
            
            # results
            model.eval()
            out_train = torch.clamp(imgn_train-model(imgn_train), 0., 1.)
            psnr_train = batch_PSNR(out_train, img_train, 1.)
            loss_train.append(loss.item())
            print(loss.item())
            PSNR_train.append(psnr_train)
            print("[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f" %
                (epoch+1, i+1, len(loader_train), loss.item(), psnr_train))
            # if you are using older version of PyTorch, you may need to change loss.item() to loss.data[0]
            if step % 10 == 0:
                # Log the scalar values
                writer.add_scalar('loss', loss.item(), step)
                writer.add_scalar('PSNR on training data', psnr_train, step)
            step += 1
        ## the end of each epoch
        model.eval()
        if((type(loss_epoch_train) == list) and (epoch == 0)):
          loss_epoch_train.append(mean(loss_train))
          PSNR_epoch_train.append(mean(PSNR_train))
          loss_epoch_train = np.asarray(loss_epoch_train)
          PSNR_epoch_train = np.asarray(PSNR_epoch_train)
        else:
          loss_epoch_train = np.append(loss_epoch_train,mean(loss_train))
          PSNR_epoch_train = np.append(PSNR_epoch_train,mean(PSNR_train))
        
        np.save(os.path.join(outf,'loss_epoch_train.npy'),loss_epoch_train)
        np.save(os.path.join(outf,'PSNR_epoch_train.npy'),PSNR_epoch_train)
        print("[on the whole epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f" %
                (epoch+1, i+1, len(loader_train), mean(loss_train),mean(PSNR_train)))
        
        # validate
        psnr_val = 0
        loss_val = 0
        for k in range(len(dataset_val)):
          img_val = dataset_val[k]
          img_val = torch.unsqueeze(img_val,0)
          noise = torch.FloatTensor(img_val.size()).normal_(mean=0, std=val_noiseL/255.)
          imgn_val = img_val + noise
          img_val, imgn_val = Variable(img_val.cuda(), volatile=True), Variable(imgn_val.cuda(), volatile=True)
          noise = Variable(noise.cuda())
          out_val=model(imgn_val)
          loss = criterion(out_val, noise) / (imgn_val.size()[0]*2)
          out_val = torch.clamp(imgn_val-model(imgn_val), 0., 1.)
          psnr_val += batch_PSNR(out_val, img_val, 1.)
          loss_val += loss.item()
          print(loss.item())
        psnr_val /= len(dataset_val)
        loss_val /= len(dataset_val)
        if((type(loss_epoch_val) == list) and (epoch == 0)):
          loss_epoch_val.append(loss_val)
          PSNR_epoch_val.append(psnr_val)
          loss_epoch_val = np.asarray(loss_epoch_val)
          PSNR_epoch_val = np.asarray(PSNR_epoch_val)
        else:
          loss_epoch_val = np.append(loss_epoch_val,loss_val)
          PSNR_epoch_val = np.append(PSNR_epoch_val,psnr_val)
        np.save(os.path.join(outf,'loss_epoch_val.npy'),loss_epoch_val)
        np.save(os.path.join(outf,'PSNR_epoch_val.npy'),PSNR_epoch_val)
        print("\n[epoch %d] PSNR_val: %.4f" % (epoch+1, psnr_val))
        print("\n[epoch %d] loss_val: %.4f" % (epoch+1, loss_val))
        writer.add_scalar('PSNR on validation data', psnr_val, epoch)
        
    
        out_train = torch.clamp(imgn_train-model(imgn_train), 0., 1.)
        # save model
        torch.save(model, os.path.join(outf , 'model_%03d.pth' % (epoch+1)))
        
	#plot graphs
 

    x=[i for i in range(1,epochs+1)]
    loss_epoch_train = list(loss_epoch_train)
    loss_epoch_val = list(loss_epoch_val)
    PSNR_epoch_train = list(PSNR_epoch_train)
    PSNR_epoch_val = list(PSNR_epoch_val)
    #plt.subplot(211)
    fig1=plt.figure()
    plt.title("loss function on both training set and validation set")
    plt.plot(x,loss_epoch_train,'-r', label='Training')
    print(len(x))
    print(len(loss_epoch_train))
    plt.plot(x,loss_epoch_val,'-b', label='Validation')
    plt.xlabel("Number of Epochs")
    plt.ylabel("Loss")
    plt.legend(loc='upper right')
    plt.draw()
    #plt.subplot(212)
    fig2=plt.figure()
    plt.title("PSNR function on both training set and validation set")
    plt.plot(x,PSNR_epoch_train,'-r',label='PSNR on the training set')
    plt.plot(x,PSNR_epoch_val,'-b',label='PSNR on the validation set')
    plt.xlabel("Number of Epochs")
    plt.ylabel("PSNR")
    plt.legend(loc='upper right')
    plt.draw()
	

if preprocess:
  if mode == 'S':
    prepare_data(data_path, patch_size=40, stride=10, aug_times=1)
  if mode == 'B':
    prepare_data(data_path, patch_size=50, stride=10, aug_times=2)
main()


# **Testing**
The hyperparameters are defined at the beginning of the code:

**num_of_layers** = 17 is the number of layers.

**logdir**       = '/content/drive/My Drive/Stereo-PyTorch-master/logs/new_training2' is the path to the diractory of the tested model.

**test_noiseL**   = 25 is the test noise level.

**data_path**   = '/content/drive/My Drive/Stereo-PyTorch-master/data' is the data path.

**name_model**    = 'model_050.pth' is the name of the tested model.



In [0]:

%matplotlib inline

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


"""defining hyperparameters"""
num_of_layers = 17
data_path     = '/content/drive/My Drive/Stereo-PyTorch-master/data'
logdir        = '/content/drive/My Drive/Stereo-PyTorch-master/logs/new_training2'
test_noiseL   = 25
name_model    = 'model_050.pth'                          

def normalize(data):
    return data/255.
  
def main():
    # Build model
    print('Loading model ...\n')
    net = DnCNN(channels=2, num_of_layers=17)
    device_ids = [0]
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    model = torch.load(os.path.join(logdir, name_model))
    #model.load_state_dict(torch.load(os.path.join(logdir, 'name_model')))
    model.eval()
    
    # load data info
    print('Loading data info ...\n')
    files_left = glob.glob(os.path.join(data_path,'test_left','*.png'))
    files_right = glob.glob(os.path.join(data_path,'test_right','*.png'))
    files_left.sort()
    files_right.sort()
    
    # process data
    psnr_test = 0
    ssim_test = 0
    for i in range(len(files_left)):
        # image
        img_right = cv2.imread(files_right[i], 0)
        img_left  = cv2.imread(files_left [i], 0)
        img_right= np.expand_dims(img_right,2)
        img_left = np.expand_dims(img_left,2)
        hv, wv, cv= img_right.shape
        img=np.zeros((2,hv,wv))
        img[0,:,:]=img_right[:,:,0]
        img[1,:,:]=img_left[:,:,0]
        img = normalize(np.float32(img[:,:,:]))
        ISource = torch.Tensor(img)
        ISource = torch.unsqueeze(ISource,0)
        
        # noise
        noise = torch.FloatTensor(ISource.size()).normal_(mean=0, std=test_noiseL/255.)
        
        # noisy image
        INoisy = ISource + noise
        ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())
        with torch.no_grad(): # this can save much memory
          imgnoisy=model(INoisy)
          Out = torch.clamp(INoisy-model(imgnoisy), 0., 1.)
          
        ## if you are using older version of PyTorch, torch.no_grad() may not be supported
        # ISource, INoisy = Variable(ISource.cuda(),volatile=True), Variable(INoisy.cuda(),volatile=True)
        # Out = torch.clamp(INoisy-model(INoisy), 0., 1.)
        
        #PSNR
        out_l = Out[:,0,:,:].unsqueeze(0)
        out_r = Out[:,1,:,:].unsqueeze(0)
        source_l = ISource[:,0,:,:].unsqueeze(0)
        source_r = ISource[:,1,:,:].unsqueeze(0)
        psnr_right = batch_PSNR(out_l,source_l,None)
        psnr_left = batch_PSNR(out_r,source_r,None)
        psnr_test += (psnr_left+psnr_right)/2

        #SSIM
        ssim_right = batch_SSIM(out_l,source_l)
        ssim_left = batch_SSIM(out_r,source_r)
        ssim_test += (ssim_left+ssim_right)/2
        
        #Preparing images for dispaly
        Out = torch.squeeze(Out)
        Out = Out.permute(1,2,0)
        image_right_c = Out[:,:,0]
        image_left_c = Out[:,:,1]
        
        
        ISource = torch.squeeze(ISource)
        ISource = ISource.permute(1,2,0)
        image_right_o = ISource[:,:,0]
        image_left_o = ISource[:,:,1]
        
      
        INoisy = torch.squeeze(INoisy)
        INoisy = INoisy.permute(1,2,0)
        image_right_n = INoisy[:,:,0]
        image_left_n = INoisy[:,:,1]
        
        #show images
        """plt.imshow(image_right_o, cmap=plt.get_cmap('gray'))
        plt.show()
        plt.imshow(image_right_n, cmap=plt.get_cmap('gray'))
        plt.show()
        plt.imshow(image_right_c, cmap=plt.get_cmap('gray'))
        plt.show()"""

        print("Image Left  ",i+1,"PSNR: ",psnr_left)
        print("Image Right ",i+1,"PSNR: ",psnr_right)
    psnr_test /= len(files_right)
    ssim_test /= len(files_right)
    print("\nPSNR on test data %f " % psnr_test)
    print("\nSSIM on test data %f " % ssim_test)


main()